                                                         ИМПОРТИРОВАНИЕ МОДУЛЕЙ

In [1]:
import scipy.stats as sps
import scipy.optimize as spo
import scipy.special as scs
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import pickle
import pyreadr

                                                        НАРЕЗКА ВЫБОРКИ НА ПОДВЫБОРКИ

In [2]:

# ТОЧКИ НАРЕЗКИ ЗАДАЮТСЯ СПИСКОМ

def cutting(data,List):        # data - список или одномерный массив,  List - список
    def ListList(N):           # создает список длины len(List), элементы которого пустые списки
        A=[]
        for i in range(N):
            A.append([])
        return(A)
    data=list(data)
    if not List[0]==0:                      # Если начальный элемент списка List не равен нулю, к результату добавляется          
        List.insert(0, 0)                   # начальный фрагмент [data[0]:data[List[0]]
    if not List[len(List)-1]==len(data):    # Если конечный элемент списка List не длине массива(списка) data, 
        List.append(len(data))              # к результату добавляется конечный фрагмент [data[List[-1]:data[len(data)]
    A=ListList(len(List)-1)
    for i in range(len(A)):
        A[i]=(data[List[i]:List[i+1]])    
    return A
####################################################################################################################################################

# НАРЕЗКА НА ПОДВЫБОРКИ ФИКСИРОВАННОЙ ДЛИНЫ

def subsamples(data,T,frombegin):             
# data - список или массив np.array
# T - объем (длина) подвыборок
# Если frombegin=True, нарезка идет от начала u отбрасывается конец, иначе - нарезка идет от конца
    data=list(data)
    n=len(data)//T                              # n - количество подвыборок
    rest=len(data)%n
    if frombegin==False:                            
        data=data[rest:(len(data))]
    else:
        data=data[0:n*T]
    List = []
    for i in range(n):
        List.append(i*T)
    Subs=cutting(data,List)
    for i in range(len(Subs)):
        Subs[i]=np.array(Subs[i])
    return Subs                                      # Подвыборки - одномерные массивы

####################################################################################################################################################
# НАРЕЗКА МАССИВА НА ПОДВЫБОРКИ ФИКСИРОВАННОЙ ДЛИНЫ

def subsamples_array(data,T,frombegin,axis):
# data - двумерный массив
# T - длина подвыборки
# axis=0 - разбиение идет по столбцам (количество строк = [исходное количество строк/T]
# axis=1 - разбиение идет по столбцам (количество столбцов = [исходное количество столбцов/T]
    N0=data.shape[0]
    N1=data.shape[1]
    if axis==0:
        n=N0//T
        SUBS=np.zeros([n,T,N1])
        for k in range(n):
            for j in range(N1):
                SUBS[k,:,j]=subsamples(data[:,j],T,True)[k]
    if axis==1:
        n=N1//T
        SUBS=np.zeros([n,N0,T])
        for k in range(n):
            for i in range(N0):
                SUBS[k,i,:]=subsamples(data[i,:],T,True)[k]
    return SUBS

                                                     ЭМПИРИЧЕСКАЯ ПЛОТНОСТЬ ВЕРОЯТНОСТИ

In [3]:
# ГИСТОГРАММА

#np.histogram(data, bins=10, range=None, density=None, weights=None)
# range  = (,) - нижнее и верхнее значение диапазона (по умолчанию (min(data),max(data))
# bins - точки разбиения диапазона (можно задать как список или числом - тогда промежутки будут равными)
# density - если density=True, гистограмма нормируется на единицу

#################################################################################################################################

# ЭМПИРИЧЕСКАЯ ПЛОТНОСТЬ
def empiric_density(data, number_breaks, mindata, maxdata):                      # data - np.array(N,);   N=data.shape[0]
    data=np.array(data)
    step=(maxdata-mindata)/number_breaks
    breaks=mindata+step*range(0,number_breaks+1)
    mids=mindata+step/2+step*range(0,number_breaks)
    density=(maxdata-mindata)/(max(data)-min(data))*np.histogram(data,bins=number_breaks,range=(mindata, maxdata),density=True)[0]
    # эмпирическая плотность отнормирована на диапазон - интеграл от плотности на диапазоне равен единице
    return([mids],[density])
##################################################################################################################################
# ИЗВЛЕКАЕТ ИЗ ФУНКЦИИ empiric_density ЧАСТОСТИ КАК np.array (n,)

def Y_density(data,number_breaks):
    return np.array(empiric_density(data, number_breaks,mindata=min(data), maxdata=max(data)))[1,0,:]
def Y_cumulative(data,number_breaks):
    return np.array(empiric_cumulative_density(data, number_breaks,mindata=min(data), maxdata=max(data)))[1,0,:]
##################################################################################################################################

# ЭМПИРИЧЕСКАЯ ПЛОТНОСТЬ И ЕЕ ГРАФИК
    # На график эмпирической плотности можно накладывать различные кривые - графики функций из списка [F]

def Empiric_density(data, number_breaks, F,colors,mindata, maxdata,MEAN):
    # data - np.array (одномерный)
    # F - список функций
    # colors - список цветов
    # mindata,maxdata - границы графика
    # MEAN - если MEAN=True, наносится вертикальная линия - среднее значение data
    Dens=empiric_density(data, number_breaks, mindata, maxdata)
    plt.figure()
    X1=list()
    Y1=list()
    
    for i in range(Dens[0][0].shape[0]):
        X1.append(np.array(Dens[0])[0,i])
        Y1.append(np.array(Dens[1])[0,i])
    Z=np.zeros([len(F),Dens[0][0].shape[0]])
    plt.plot(X1,Y1,'b')
    for i in range(len(F)):
        Z[i,:]=np.vectorize(F[i])(X1)
        plt.plot(X1,Z[i],color=colors[i])
    plt.title('EMPIRIC DENSITY')
    plt.xlabel('breaks')
    plt.ylabel('probabilities')
    plt.grid()
    plt.axvline(lw=3,color='k',x=0)
    plt.axhline(lw=3,color='k',y=0)
    if MEAN==True:
        plt.axvline(lw=3,color='g',x=np.mean(data))
    return plt.show

                                                         ЭМПИРИЧЕСКАЯ ФУНКЦИЯ РАСПРЕДЕЛЕНИЯ

In [4]:
# ЭМПИРИЧЕСКАЯ ФУНКЦИЯ РАСПРЕДЕЛЕНИЯ
def empiric_cumulative_density(t,data,less):
    if t<=min(data):
        return 0
    if t>=max(data):
        return 1
    data=sorted(list(data))
    if t<data[0]:
        return 0
    leftx=list(filter(lambda x: x<=t, data))[-1]
    rightx=list(filter(lambda x: x>t, data))[0]
    left_index=data.index(leftx)
    right_index=data.index(rightx)
    if less==True:
        leftF=left_index/len(data)
        rightF=right_index/len(data)
    else:
        leftF=(left_index+1)/len(data)
        rightF=(right_index+1)/len(data)
    Fdistrx=leftF+(rightF-leftF)/(rightx-leftx)*(t-leftx)
    return Fdistrx


# ЭМПИРИЧЕСКАЯ ФУНКЦИЯ РАСПРЕДЕЛЕНИЯ И ЕЕ ГРАФИК
    # На график эмпирической плотности можно накладывать различные кривые - графики функций из списка [F]

def Empiric_cumulative_density(data,number_breaks,mindata,maxdata,F,colors,less,MEAN):
    step=(maxdata-mindata)/number_breaks
    X1=np.zeros(number_breaks)
    Y1=np.zeros(number_breaks)
    for i in range(number_breaks):
        X1[i]=min(data)+i*step
        Y1[i]=empiric_cumulative_density(X1[i],data,less)
    Z=np.zeros([len(F),X1.shape[0]])
    plt.plot(X1,Y1,'b')
    for i in range(len(F)):
        Z[i,:]=np.vectorize(F[i])(X1)
        plt.plot(X1,Z[i],color=colors[i])
    plt.title('EMPIRIC DENSITY')
    plt.xlabel('breaks')
    plt.ylabel('probabilities')
    plt.grid()
    plt.axvline(lw=3,color='k',x=0)
    plt.axhline(lw=3,color='k',y=0)
    if MEAN==True:
        plt.axvline(lw=3,color='g',x=np.mean(data))
    return plt.show()

#######################################################################################################################################

                                                             СТАТИСТИЧЕСКИЙ АНАЛИЗ ТЕСТА

In [5]:
def pval_stat1(A,alpha):
    B=np.zeros(A.shape[0])
    for i in range(A.shape[0]):
        if A[i]>alpha:
            B[i]=1
        else:
            B[i]=0
    count=np.sum(B)/A.shape[0]
    return count

def pval_stat2(A,alpha,axis):
    B=np.zeros([A.shape[0],A.shape[1]])
    for i in range(A.shape[0]):
        for j in range(A.shape[1]):
            if A[i,j]>alpha:
                B[i,j]=1
            else:
                B[i,j]=0
    count=np.sum(B,axis=axis)/A.shape[axis]
    return count

                                                   ЭКСПЕРИМЕНТЫ С РЕАЛЬНЫМИ ДАННЫМИ

In [6]:
                                                  П р и м е р 1_Доходности финансовых активов

SyntaxError: invalid decimal literal (1475417063.py, line 1)

In [8]:
DJ=pyreadr.read_r("DJ_COMPONENTS.RData")         #  словарь, полученный из данных R
CLOSE_DAILY_DJ=DJ['CLOSE_DAILY']                                        #  DataFrame цен, полученная по выбору ключа словаря
PRICES_DJ=CLOSE_DAILY_DJ.to_numpy()                                     #  np-array цен
RET_DJ=np.diff(np.log(PRICES_DJ),axis=0)                                #  Доходности
RET_DJ

array([[ 0.09607481,  0.12260232,  0.00903167, ...,  0.10001484,
         0.26764945,  0.02784054],
       [ 0.04112798,  0.02656703, -0.02529219, ...,  0.01037623,
         0.10886875, -0.05038102],
       [-0.04112798, -0.04207121, -0.03758271, ...,  0.01409376,
        -0.16218166,  0.01508324],
       ...,
       [ 0.02171147,  0.03505865, -0.00649738, ...,  0.0139811 ,
         0.03121484, -0.00930725],
       [-0.02651898, -0.07227494, -0.05056137, ..., -0.02173293,
         0.00099704, -0.003684  ],
       [ 0.01256388,  0.00441066,  0.02768377, ...,  0.00936319,
        -0.02457709,  0.006831  ]])

                                                      Тест Колмогорова-Смирнова

In [16]:
T_RET=100
RET_DJ_SUBS=subsamples_array(RET_DJ,T=T_RET,frombegin=True,axis=0)


In [17]:
RET_KS=np.zeros([RET_DJ_SUBS.shape[0],RET_DJ_SUBS.shape[2]])
for k in range(RET_DJ_SUBS.shape[0]):
    for j in range(RET_DJ_SUBS.shape[2]):
        RET_KS[k,j]=sps.ks_2samp(RET_DJ_SUBS[k,0:round(T_RET/2),j],RET_DJ_SUBS[k,round(T_RET/2):T_RET,j]).pvalue
pd_RET_KS=pd.DataFrame(RET_KS)
pd_RET_KS.columns=CLOSE_DAILY_DJ.columns
pd_RET_KS

,AAPL,AMD,BA,CSCO,GS,IBM,INTC,JPM,KO,MSFT,NVDA,SBUX
0,0.869262,0.548685,0.271914,0.548685,0.548685,0.869262,0.997711,0.999995,0.395940,0.716647,0.716647,0.548685
1,0.869262,0.716647,0.271914,0.548685,0.271914,0.869262,0.716647,0.966746,0.716647,0.548685,0.716647,0.716647
2,0.548685,0.039195,0.966746,0.548685,0.716647,0.178587,0.112385,0.716647,0.178587,0.178587,0.112385,0.716647
3,0.112385,0.395940,0.869262,0.869262,0.395940,0.112385,0.067795,0.112385,0.178587,0.869262,0.548685,0.112385
4,0.869262,0.716647,0.178587,0.395940,0.716647,0.112385,0.548685,0.067795,0.548685,0.395940,0.716647,0.395940
5,0.548685,0.271914,0.178587,0.271914,0.178587,0.997711,0.869262,0.039195,0.039195,0.395940,0.869262,0.716647
6,0.716647,0.021708,0.395940,0.548685,0.548685,0.112385,0.716647,0.548685,0.395940,0.716647,0.548685,0.395940
7,0.395940,0.966746,0.395940,0.548685,0.716647,0.395940,0.548685,0.548685,0.067795,0.271914,0.548685,0.997711
8,0.869262,0.548685,0.716647,0.966746,0.997711,0.869262,0.869262,0.395940,0.112385,0.112385,0.395940,0.716647
9,0.869262,0.716647,0.869262,0.271914,0.548685,0.178587,0.178587,0.966746,0.395940,0.869262,0.112385,0.716647


In [18]:
alpha_ret=0.1
KS_ret_stat=pval_stat2(RET_KS,alpha_ret,axis=0)
KS_ret_stat                    # частость принятия гипотезы

array([0.89285714, 0.80357143, 0.91071429, 0.85714286, 0.94642857,
       0.98214286, 0.83928571, 0.83928571, 0.89285714, 0.92857143,
       0.91071429, 0.96428571])

                                            Тест на равенство математических ожиданий

In [19]:
RET_STUD=np.zeros([RET_DJ_SUBS.shape[0],RET_DJ_SUBS.shape[2]])
Mean1_RET=np.zeros([RET_DJ_SUBS.shape[0],RET_DJ_SUBS.shape[2]])
Mean2_RET=np.zeros([RET_DJ_SUBS.shape[0],RET_DJ_SUBS.shape[2]])
Std1_RET=np.zeros([RET_DJ_SUBS.shape[0],RET_DJ_SUBS.shape[2]])
Std2_RET=np.zeros([RET_DJ_SUBS.shape[0],RET_DJ_SUBS.shape[2]])

for k in range(RET_DJ_SUBS.shape[0]):
    Mean1_RET[k,:]=np.mean(RET_DJ_SUBS[k,0:round(T_RET/2),:],axis=0)
    Mean2_RET[k,:]=np.mean(RET_DJ_SUBS[k,round(T_RET/2):T_RET,:],axis=0)
    Std1_RET[k,:]=np.std(RET_DJ_SUBS[k,0:round(T_RET/2),:],axis=0)
    Std2_RET[k,:]=np.std(RET_DJ_SUBS[k,round(T_RET/2):T_RET,:],axis=0)
    

for k in range(RET_DJ_SUBS.shape[0]):
    for j in range(RET_DJ_SUBS.shape[2]):
     RET_STUD[k,j]=sps.ttest_ind_from_stats(mean1=Mean1_RET[k,j], std1=Std1_RET[k,j], nobs1=T_RET, mean2=Mean2_RET[k,j], 
                                         std2=Std2_RET[k,j], nobs2=T_RET, equal_var=False, alternative='two-sided').pvalue
pd_RET_STUD=pd.DataFrame(RET_STUD)
pd_RET_STUD.columns=CLOSE_DAILY_DJ.columns
pd_RET_STUD

,AAPL,AMD,BA,CSCO,GS,IBM,INTC,JPM,KO,MSFT,NVDA,SBUX
0,0.656956,0.647459,0.162150,0.188986,0.333047,0.682308,0.753431,0.849937,0.050960,0.774461,0.580170,0.601062
1,0.693020,0.996458,0.056827,0.940341,0.409000,0.416734,0.305947,0.737573,0.132826,0.881768,0.901256,0.660757
2,0.330079,0.002857,0.997322,0.054653,0.127556,0.007955,0.017213,0.372924,0.195015,0.089970,0.009274,0.325342
3,0.070995,0.979334,0.841210,0.622624,0.219949,0.484565,0.361421,0.268312,0.208289,0.731156,0.056032,0.301285
4,0.778148,0.193340,0.038936,0.126227,0.731197,0.947182,0.454483,0.051389,0.968371,0.713815,0.866190,0.144961
5,0.272761,0.115573,0.012985,0.517745,0.233993,0.761064,0.957164,0.002475,0.161098,0.863926,0.329280,0.132905
6,0.785310,0.007670,0.379023,0.514719,0.770793,0.068875,0.579543,0.432218,0.914756,0.571633,0.961209,0.687823
7,0.035819,0.681331,0.084983,0.198790,0.448693,0.671402,0.213912,0.101944,0.009532,0.737797,0.028963,0.562578
8,0.400438,0.438351,0.872282,0.890997,0.421310,0.424662,0.254292,0.078655,0.063071,0.564003,0.058826,0.174236
9,0.883207,0.205520,0.511797,0.307364,0.826415,0.015763,0.053948,0.456043,0.017794,0.539439,0.110763,0.136444


                                 Сравнение результатов тестов Колмогорова-Смирнова и Стьюдента_Уэлша

In [20]:

STUD_KS=np.zeros([RET_DJ_SUBS.shape[0],RET_DJ_SUBS.shape[2]])
for k in range(RET_DJ_SUBS.shape[0]):
    for j in range(RET_DJ_SUBS.shape[2]):
        if RET_KS[k,j]>alpha_ret and RET_STUD[k,j]<alpha_ret:
            STUD_KS[k,j]=1
        else:
            STUD_KS[k,j]=0

KS_STUD=np.zeros([RET_DJ_SUBS.shape[0],RET_DJ_SUBS.shape[2]])
for k in range(RET_DJ_SUBS.shape[0]):
    for j in range(RET_DJ_SUBS.shape[2]):
        if RET_KS[k,j]<alpha_ret and RET_STUD[k,j]>alpha_ret:
            STUD_KS[k,j]=1
        else:
            STUD_KS[k,j]=0
[STUD_KS,KS_STUD]

[array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0

                                                        П р и м е р 2_Тестирование медпрепарата

In [21]:
MED1=pyreadr.read_r("MED1.RData")
MED1

OrderedDict([('polyps',
                 participant_id     sex   age  baseline treatment  number3m  number12m
              0             001  female  17.0         7  sulindac         6        NaN
              1             002  female  20.0        77   placebo        67       63.0
              2             003    male  16.0         7  sulindac         4        2.0
              3             004  female  18.0         5   placebo         5       28.0
              4             005    male  22.0        23  sulindac        16       17.0
              5             006  female  13.0        35   placebo        31       61.0
              6             007  female  23.0        11  sulindac         6        1.0
              7             008    male  34.0        12   placebo        20        7.0
              8             009    male  50.0         7   placebo         7       15.0
              9             010    male  19.0       318   placebo       347       44.0
              10   